<a href="https://colab.research.google.com/github/meekmarcelin/language-translation/blob/main/language-translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, TimeDistributed, Bidirectional, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Data collection and preprocessing
english_sentences = ["hello", "how are you", "good morning"]
yoruba_sentences = ["bawo ni", "bawo ni o se wa", "e kaaro"]

# Tokenization
eng_tokenizer = Tokenizer()
yor_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
yor_tokenizer.fit_on_texts(yoruba_sentences)

eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
yor_sequences = yor_tokenizer.texts_to_sequences(yoruba_sentences)

# Padding
max_eng_len = max(len(seq) for seq in eng_sequences)
max_yor_len = max(len(seq) for seq in yor_sequences)

eng_sequences = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
yor_sequences = pad_sequences(yor_sequences, maxlen=max_yor_len, padding='post')

# Splitting the data
eng_train, eng_val, yor_train, yor_val = train_test_split(eng_sequences, yor_sequences, test_size=0.2)

# Model development
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_dim=len(eng_tokenizer.word_index)+1, output_dim=embedding_dim)(encoder_inputs)
encoder_output, forward_h, backward_h = Bidirectional(GRU(units, return_state=True))(enc_emb)

# Concatenate the forward and backward states
state_h = Concatenate()([forward_h, backward_h])

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(input_dim=len(yor_tokenizer.word_index)+1, output_dim=embedding_dim)(decoder_inputs)
dec_gru = GRU(units * 2, return_sequences=True, return_state=True)
decoder_output, _ = dec_gru(dec_emb, initial_state=state_h)
decoder_dense = TimeDistributed(Dense(len(yor_tokenizer.word_index)+1, activation='softmax'))
decoder_outputs = decoder_dense(decoder_output)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
model.fit([eng_train, yor_train[:, :-1]], yor_train[:, 1:, np.newaxis],
          validation_data=([eng_val, yor_val[:, :-1]], yor_val[:, 1:, np.newaxis]),
          epochs=50, batch_size=64)

# Save the model
model.save('eng_to_yor_translation_model.h5')

# Evaluation
def evaluate_model(input_sentence):
    input_seq = eng_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_eng_len, padding='post')
    states = model.layers[3].predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = yor_tokenizer.word_index['bawo']  # Use actual start token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = model.layers[5].predict([target_seq] + [states])

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = yor_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        if sampled_word == 'end' or len(decoded_sentence.split()) > max_yor_len:
            stop_condition = True

        target_seq



Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 2.0786 - accuracy: 0.0000e+00 - val_loss: 1.9730 - val_accuracy: 0.7500
Epoch 2/50
1/1 [==============================] - 0s 230ms/step - loss: 1.9366 - accuracy: 0.7500 - val_loss: 1.8525 - val_accuracy: 0.7500
Epoch 3/50
1/1 [==============================] - 0s 232ms/step - loss: 1.7750 - accuracy: 0.7500 - val_loss: 1.6795 - val_accuracy: 0.7500
Epoch 4/50
1/1 [==============================] - 0s 217ms/step - loss: 1.5623 - accuracy: 0.5000 - val_loss: 1.4242 - val_accuracy: 0.7500
Epoch 5/50
1/1 [==============================] - 0s 261ms/step - loss: 1.2993 - accuracy: 0.5000 - val_loss: 1.1062 - val_accuracy: 0.7500
Epoch 6/50
1/1 [==============================] - 0s 257ms/step - loss: 1.0911 - accuracy: 0.5000 - val_loss: 0.9151 - val_accuracy: 0.7500
Epoch 7/50
1/1 [==============================] - 0s 229ms/step - loss: 0.9542 - accuracy: 0.5000 - val_loss: 0.8773 - val_accuracy: 0.7500
Epoch 8/50
1/1 [===

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
